In [2]:
import numpy as np
import torch
from torch import nn 

In [3]:
# 인접행렬 생성 

mat = np.array([[0,0,1,0,0,0,0],[1,0,0,0,0,0,0],[0,1,1,0,0,0,0],[1,0,1,0,0,0,0],[0,1,0,0,0,0,0],[0,0,0,1,0,0,0],[0,0,0,1,1,0,0],[0,0,0,0,1,0,0],[0,0,0,0,0,0,1],[0,0,0,0,0,1,0]])
print(mat)
print(mat.shape)

[[0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0]
 [0 1 1 0 0 0 0]
 [1 0 1 0 0 0 0]
 [0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 1 1 0 0]
 [0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
(10, 7)


In [5]:
#아래 코드를 좀 변형하면 될듯?
class MaskedLinear(nn.Module):
    def __init__(self, in_dim, out_dim, indices_mask):
        """
       :param in_features: number of input features
       :param out_features: number of output features
       :param indices_mask: list of two lists containing indices for dimensions 0 and 1, used to create the mask
       """
        super(MaskedLinear, self).__init__()
        
 
        def backward_hook(grad):
            # Clone due to not being allowed to modify in-place gradients
            out = grad.clone()
            out[self.mask] = 0
            return out
 
        self.linear = nn.Linear(in_dim, out_dim).cuda()
        self.mask = torch.ones([out_dim, in_dim]).byte().cuda()
        self.mask[indices_mask] = 0 # create mask
        self.linear.weight.data[self.mask] = 0 # zero out bad weights
        self.linear.weight.register_hook(backward_hook) # hook to zero out bad gradients
 
    def forward(self, input):
        return self.linear(input)
    
    
